<a href="https://colab.research.google.com/github/ALXAVIER-DEV/Spark/blob/master/Aula_4_Introdu%C3%A7%C3%A3o_aos_RDDs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# **Running Pyspark in Colab**

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.0.1 with hadoop 2.7 and Java 8. The tools installation can be carried out inside the Jupyter Notebook of the Colab. One important note is that if you are new in Spark, it is better to avoid Spark 2.4.0 version since some people have already complained about its compatibility issue with python. 
Follow the steps to install the dependencies:

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!pip install pyspark

Now that you installed Spark and Java in Colab, it is time to set the environment path which enables you to run Pyspark in your Colab environment. Set the location of Java and Spark by running the following code:

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

Run a local spark session to test your installation:

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
spark

# Reading a CSV from google drive

Utilizando o Google Colab, é possível importar os datasets diretamente do Google Drive, sem ter que realizar o upload manual dos mesmos para a instância colab manualmente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
spark.read\
  .option("inferSchema", "true") \
  .option("header", "true") \
  .option("delimiter", ",") \
  .csv("drive/My\ Drive/My\ Professional\ Carrer/Spark\ course/virtual_classroom/colab_test/test.csv") \
  .show()

# Introdução aos RDDs
Nessa aula veremos o que são os RDDs ou Resilient Distributed Datasets e como interagir com algumas operações básicas sobre os mesmos

### RDD
RDD ou Resilient Distributed Datasets são estruturas distribuídas e imutáveis de objetos Python, Scala, Java ou qualquer que seja a linguagem que o usuário esteja operando os RDDs. Ao contrário dos Dataframes que são estruturados no conceito de Row com um schema conhecido, os RDDs são genéricos deixando a cargo do programador o seu uso e a sua estruturação

### Low-level APIs
São APIs baixo nível fornecidas pelo Spark para manipular as estruturas elementares do Spark: os **RDDs**, **Shared Variables** e **Accumulators**. Normalmente o usuário opta por usar as Low-level APIs em 3 situações:
- Quando é necessário ter o controle explícito de como o dado é distribuído ao longo do cluster
- Manter código legado escrito usando RDDs
- Acessar variáveis compartilhadas no cluster como um todo

Mesmo que você seja avançado em programação com RDDs, é recomendado que você siga utilizando as APIs High-level, a menos que se depare com uma das 3 situações acima. Ao utilizar os RDDs, assume-se que todo o controle está na mão do usuário e este pode não saber ao certo o que está fazendo e danificar a performance da aplicação.

É importante ressaltar que a API High-level Dataframe é inteiramente baseada nos RDDs, ou seja, toda transformação realizada em cima dos Dataframes, se torna uma transformação primitiva em RDD e entender como funciona essas APIs pode fazer a diferença na construção de aplicações com mais performance e robustez.

### Spark Context
É o ponto de entrada para se trabalhar com as Low-level APIs. Está para os RDDs assim como o Spark Session está para os Dataframes

In [ ]:
sc = spark.sparkContext
sc

### Criando RDDs
Ao criar um RDD, é possível criá-lo de 3 formas: 
- a partir de coleções locais em Python
- a partir de data sources primitivas
- a partir de Dataframes

##### Utilizando coleções locais
Ao utilizar coleções locais, temos o comando `sc.parallelize()` que recebe como parâmetro a **coleção** e o número de **partições**

In [ ]:
text = "Spark The Definitive Guide : Big Data Processing Made Simple"
collection = text.split(" ")

In [ ]:
type(collection)

list

In [ ]:
rdd_words = sc.parallelize(collection, 2)
rdd_words.setName("myWords")
rdd_words

myWords ParallelCollectionRDD[5] at readRDDFromFile at PythonRDD.scala:262

In [ ]:
rdd_words.collect()

['Spark',
 'The',
 'Definitive',
 'Guide',
 ':',
 'Big',
 'Data',
 'Processing',
 'Made',
 'Simple']

In [ ]:
data_dict = [{
    "nome":"Jose",
    "idade": 52,
    "profissao": "Advogado"
},{
    "nome":"Maria",
    "idade": 35,
    "profissao": "Médico",
    "sexo": "feminino"
},{
    "nome":"Pedro",
    "idade": 25,
    "profissao": "Estudante"
},
print,
True]

data_dict

[{'idade': 52, 'nome': 'Jose', 'profissao': 'Advogado'},
 {'idade': 35, 'nome': 'Maria', 'profissao': 'Médico', 'sexo': 'feminino'},
 {'idade': 25, 'nome': 'Pedro', 'profissao': 'Estudante'},
 <function print>,
 True]

In [ ]:
rdd_dict = sc.parallelize(data_dict, 2)
rdd_dict.collect()

[{'idade': 52, 'nome': 'Jose', 'profissao': 'Advogado'},
 {'idade': 35, 'nome': 'Maria', 'profissao': 'Médico', 'sexo': 'feminino'},
 {'idade': 25, 'nome': 'Pedro', 'profissao': 'Estudante'},
 <function print>,
 True]

##### Utilizando arquivos como data source
Ao utilizar arquivos de texto, cada linha do arquivo será uma linha do RDD.

In [ ]:
data_dir = "vgsales.csv"

rdd = sc.textFile(data_dir)
rdd.setName("vgsales_csv")

vgsales_csv MapPartitionsRDD[15] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:
rdd.take(10)

['Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales',
 '1,Wii Sports,Wii,2006,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74',
 '2,Super Mario Bros.,NES,1985,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24',
 '3,Mario Kart Wii,Wii,2008,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82',
 '4,Wii Sports Resort,Wii,2009,Sports,Nintendo,15.75,11.01,3.28,2.96,33',
 '5,Pokemon Red/Pokemon Blue,GB,1996,Role-Playing,Nintendo,11.27,8.89,10.22,1,31.37',
 '6,Tetris,GB,1989,Puzzle,Nintendo,23.2,2.26,4.22,0.58,30.26',
 '7,New Super Mario Bros.,DS,2006,Platform,Nintendo,11.38,9.23,6.5,2.9,30.01',
 '8,Wii Play,Wii,2006,Misc,Nintendo,14.03,9.2,2.93,2.85,29.02',
 '9,New Super Mario Bros. Wii,Wii,2009,Platform,Nintendo,14.59,7.06,4.7,2.26,28.62']

### Interoperabilidade entre Dataframes e RDDs
As APIs do Spark fornecem métodos para realizar a conversão de Dataframes em RDDs e vice versa. Para isso, temos dois métodos importantes: `rdd` e `toDF()`

##### Convertendo um Dataframe para um RDD

In [ ]:
df = spark.read.load(data_dir, format="csv", inferSchema="true", header="true")

In [ ]:
df.rdd.take(5)

[Row(Rank=1, Name='Wii Sports', Platform='Wii', Year='2006', Genre='Sports', Publisher='Nintendo', NA_Sales=41.49, EU_Sales=29.02, JP_Sales=3.77, Other_Sales=8.46, Global_Sales=82.74),
 Row(Rank=2, Name='Super Mario Bros.', Platform='NES', Year='1985', Genre='Platform', Publisher='Nintendo', NA_Sales=29.08, EU_Sales=3.58, JP_Sales=6.81, Other_Sales=0.77, Global_Sales=40.24),
 Row(Rank=3, Name='Mario Kart Wii', Platform='Wii', Year='2008', Genre='Racing', Publisher='Nintendo', NA_Sales=15.85, EU_Sales=12.88, JP_Sales=3.79, Other_Sales=3.31, Global_Sales=35.82),
 Row(Rank=4, Name='Wii Sports Resort', Platform='Wii', Year='2009', Genre='Sports', Publisher='Nintendo', NA_Sales=15.75, EU_Sales=11.01, JP_Sales=3.28, Other_Sales=2.96, Global_Sales=33.0),
 Row(Rank=5, Name='Pokemon Red/Pokemon Blue', Platform='GB', Year='1996', Genre='Role-Playing', Publisher='Nintendo', NA_Sales=11.27, EU_Sales=8.89, JP_Sales=10.22, Other_Sales=1.0, Global_Sales=31.37)]

##### Convertendo um RDD para um Dataframe

In [ ]:
rdd = sc.textFile(data_dir)
rdd

vgsales.csv MapPartitionsRDD[36] at textFile at NativeMethodAccessorImpl.java:0

In [ ]:
rdd \
    .map(lambda x: (x,)) \
    .toDF() \
    .show()

+--------------------+
|                  _1|
+--------------------+
|Rank,Name,Platfor...|
|1,Wii Sports,Wii,...|
|2,Super Mario Bro...|
|3,Mario Kart Wii,...|
|4,Wii Sports Reso...|
|5,Pokemon Red/Pok...|
|6,Tetris,GB,1989,...|
|7,New Super Mario...|
|8,Wii Play,Wii,20...|
|9,New Super Mario...|
|10,Duck Hunt,NES,...|
|11,Nintendogs,DS,...|
|12,Mario Kart DS,...|
|13,Pokemon Gold/P...|
|14,Wii Fit,Wii,20...|
|15,Wii Fit Plus,W...|
|16,Kinect Adventu...|
|17,Grand Theft Au...|
|18,Grand Theft Au...|
|19,Super Mario Wo...|
+--------------------+
only showing top 20 rows



In [ ]:
data_dict = [{
    "nome":"Jose",
    "idade": 52,
    "profissao": "Advogado",
    "sexo": 1
},{
    "nome":"Maria",
    "idade": 35,
    "profissao": "Médico",
    "sexo": "feminino"
},{
    "nome":"Pedro",
    "idade": 25,
    "profissao": "Estudante"
}]

data_dict

[{'idade': 52, 'nome': 'Jose', 'profissao': 'Advogado', 'sexo': 1},
 {'idade': 35, 'nome': 'Maria', 'profissao': 'Médico', 'sexo': 'feminino'},
 {'idade': 25, 'nome': 'Pedro', 'profissao': 'Estudante'}]

In [ ]:
rdd = sc.parallelize(data_dict)
rdd.collect()

[{'idade': 52, 'nome': 'Jose', 'profissao': 'Advogado', 'sexo': 1},
 {'idade': 35, 'nome': 'Maria', 'profissao': 'Médico', 'sexo': 'feminino'},
 {'idade': 25, 'nome': 'Pedro', 'profissao': 'Estudante'}]

In [ ]:
rdd.toDF().show()

/usr/local/lib/python3.6/dist-packages/pyspark/sql/session.py:401: UserWarning: Using RDD of dict to inferSchema is deprecated. Use pyspark.sql.Row instead
  warnings.warn("Using RDD of dict to inferSchema is deprecated. "


+-----+-----+---------+----+
|idade| nome|profissao|sexo|
+-----+-----+---------+----+
|   52| Jose| Advogado|   1|
|   35|Maria|   Médico|null|
|   25|Pedro|Estudante|null|
+-----+-----+---------+----+

